In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import networkx as nx
from pyvis.network import Network
from IPython.display import display, HTML

In [8]:
import requests

url = "http://localhost:9000"

sparql_query = """
PREFIX ub: <http://swat.cse.lehigh.edu/onto/univ-bench.owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT ?professor ?course WHERE {
    ?professor rdf:type ub:Professor .
    ?professor ub:teacherOf ?course .
}
"""

payload = {
    "username": "root",
    "password": "123456",
    "db_name": "lubm",
    "operation": "query",
    "format": "json",
    "sparql": sparql_query
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
print(response.json())


{'StatusCode': 1005, 'StatusMsg': 'Database not load yet.'}


In [21]:
import requests
from pyvis.network import Network

# gStore 接口地址
url = "http://localhost:9000"
label_uri = "http://www.wikidata.org/entity/Q1047508"  # Interscope Records

# SPARQL 查询
sparql_query = f"""
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?artist ?artistLabel ?work ?workLabel ?coArtist ?coArtistLabel
WHERE {{
  ?artist wdt:P264 <{label_uri}> .
  ?work wdt:P175 ?artist .
  OPTIONAL {{
    ?work wdt:P175 ?coArtist .
    FILTER(?coArtist != ?artist)
  }}
  OPTIONAL {{ ?artist rdfs:label ?artistLabel . FILTER(LANG(?artistLabel) = "en") }}
  OPTIONAL {{ ?work rdfs:label ?workLabel . FILTER(LANG(?workLabel) = "en") }}
  OPTIONAL {{ ?coArtist rdfs:label ?coArtistLabel . FILTER(LANG(?coArtistLabel) = "en") }}
}}
LIMIT 100
"""

payload = {
    "username": "root",
    "password": "123456",
    "db_name": "wikidata_music",
    "operation": "query",
    "format": "json",
    "sparql": sparql_query
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
data = response.json()

# 打印结果以供检查
print(data)

{'head': {'link': [], 'vars': ['artist', 'artistLabel', 'work', 'workLabel', 'coArtist', 'coArtistLabel']}, 'results': {'bindings': []}, 'StatusCode': 0, 'StatusMsg': 'success', 'AnsNum': 0, 'OutputLimit': 100, 'ThreadId': '139815928891072', 'QueryTime': '1'}


In [37]:
from pyvis.network import Network
import requests

# gStore 查询接口
url = "http://localhost:9000"
label_uri = "http://www.wikidata.org/entity/Q21077"  # Warner Music Group

# SPARQL 查询语句（不含共同艺人）
sparql_query = f"""
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?artist ?artistLabel ?work ?workLabel
WHERE {{
  ?artist wdt:P264 <{label_uri}> .
  ?work wdt:P175 ?artist .
  OPTIONAL {{ ?artist rdfs:label ?artistLabel . FILTER(LANG(?artistLabel) = "en") }}
  OPTIONAL {{ ?work rdfs:label ?workLabel . FILTER(LANG(?workLabel) = "en") }}
}}
LIMIT 50
"""

payload = {
    "username": "root",
    "password": "123456",
    "db_name": "wikidata_music",
    "operation": "query",
    "format": "json",
    "sparql": sparql_query
}

headers = {"Content-Type": "application/json"}

# 请求数据
try:
    response = requests.post(url, json=payload, headers=headers)
    results = response.json()
except Exception as e:
    print(f"❌ 请求失败：{e}")
    results = {"results": {"bindings": []}}

# 初始化 PyVis 网络图
# 初始化 PyVis 网络图
net = Network(height="800px", width="100%", bgcolor="#1e1e1e", font_color="white")
net.force_atlas_2based(gravity=-25, central_gravity=0.02, spring_length=200, spring_strength=0.05)

added_nodes = set()
bindings = results.get("results", {}).get("bindings", [])

if not bindings:
    print("⚠ 查询结果为空，未生成任何图谱。")

# 添加厂牌节点
net.add_node(label_uri, label="Warner Music Group", title=label_uri,
             color="#ff6666", shape="triangle", size=30)
added_nodes.add(label_uri)

# 构建图谱
for row in bindings:
    artist_uri = row["artist"]["value"]
    artist_label = row.get("artistLabel", {}).get("value", artist_uri)

    work_uri = row["work"]["value"]
    work_label = row.get("workLabel", {}).get("value", work_uri)

    # 添加艺术家节点
    if artist_uri not in added_nodes:
        net.add_node(artist_uri, label=artist_label, title=artist_uri,
                     color="#6fa8dc", shape="dot", size=20)
        added_nodes.add(artist_uri)

    # 添加作品节点
    if work_uri not in added_nodes:
        net.add_node(work_uri, label=work_label, title=work_uri,
                     color="#f6b26b", shape="box", size=15)
        added_nodes.add(work_uri)

    # 边连接
    net.add_edge(label_uri, artist_uri, title="signed with", color="#bbbbbb")
    net.add_edge(artist_uri, work_uri, title="performed", color="#ffffff")

# 展示图谱（改为 write_html）
try:
    output_file = "music_graph_simple.html"
    net.write_html(output_file)
    print(f"✅ 图谱 HTML 已生成：{output_file}（请手动用浏览器打开）")
except Exception as e:
    print(f"❌ 图谱生成失败：{e}")

✅ 图谱 HTML 已生成：music_graph_simple.html（请手动用浏览器打开）
